# Pre


In [ ]:
! pip install -U pandasql
import torch
import pandas as pd
import pandasql as ps
import numpy as np

In [2]:
from google.colab import drive 

In [ ]:
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/tarak/Players-Block-20211116T025030Z-001/Players-Block/GNN/data/

In [5]:
! ls

 Competes_in_edges.csv	  gnn_data.pdf	        Plays_with_edges.csv
 Comp_season_nodes.csv	  Hosts_edges.csv      'Team_nodes(1).csv'
 Field_player_nodes.csv   Manager_nodes.csv     Team_nodes.csv
 GK_nodes.csv		  Manages_edges.csv
 gnn_data.docx		  Plays_for_edges.csv


In [6]:
PlaysWithEdgesDf = pd.read_csv('./Plays_with_edges.csv')
CompetesInEdgesDf = pd.read_csv('./Competes_in_edges.csv')
CompSeasonNodesDf = pd.read_csv ('Comp_season_nodes.csv')
FieldPlayerNodesDf = pd.read_csv('Field_player_nodes.csv')
GKNodesDf = pd.read_csv('GK_nodes.csv')
HostsEdgesDf = pd.read_csv('Hosts_edges.csv')
ManagerNodesDf = pd.read_csv('Manager_nodes.csv')
ManagesEdgesDf = pd.read_csv('Manages_edges.csv')
PlaysForEdgesDf = pd.read_csv('Plays_for_edges.csv')
TeamNodesDf = pd.read_csv('Team_nodes.csv')


# Go

In [84]:
query = """ 
  select *
  from TeamNodesDf T inner join HostsEdgesDf H
    on T.nodeName = H.dest_nodeName
  order by gameDate, gameId
"""
away_features = ps.sqldf(query).loc[:,
                                    [ 'preGame_steph',
                                     'preGame_quality_overall',
                                     'preGame_quality_attack',
                                     'preGame_quality_midfield',
                                     'preGame_quality_defence',
                                     'preGame_proportion_pts']
                ]
query = """ 
  select *
  from TeamNodesDf T inner join HostsEdgesDf H
    on T.nodeName = H.src_nodeName
  order by gameDate, gameId
"""
home_features = ps.sqldf(query).loc[:,
                    [ 'preGame_steph',
                      'preGame_quality_overall',
                      'preGame_quality_attack',
                      'preGame_quality_midfield',
                      'preGame_quality_defence',
                      'preGame_proportion_pts']
]
ps.sqldf(query)
results = ps.sqldf(query).loc[:, ['postGame_tgt_gd']]

In [85]:
results = torch.from_numpy(results.values).view(-1)
dummy = 6
results[results > 0] = dummy + 2
results[results == 0] = dummy + 1
results[results < 0] = dummy
results -= dummy

home_features = torch.from_numpy(home_features.values)
away_features = torch.from_numpy(away_features.values)

# Blade Chest Model

In [53]:
from torch.nn import Module, Linear, NLLLoss, Tanh, BatchNorm1d, Dropout, LogSoftmax, PairwiseDistance
from torch.optim import Adam

In [87]:
class BladeChest(Module):
  def __init__(self, feature_size, hidden_size, dropout_rate=0.5):
    super(BladeChest, self).__init__()
    self.feature_size = feature_size
    self.hidden_size = hidden_size
    self.blade_transfrom = Linear(self.feature_size, self.hidden_size)
    self.chest_transform = Linear(self.feature_size, self.hidden_size)
    self.activation = Tanh()
    self.feature_bn = BatchNorm1d(self.feature_size)
    self.hidden_bn = BatchNorm1d(self.hidden_size)
    self.regularizer = Dropout(dropout_rate)
    self.result_transform = Linear(1, 3)
    self.distance_fn = PairwiseDistance()
    self.classifier = LogSoftmax(-1)

  def _matchup_score(self, away_blade, home_chest, home_blade, away_chest):
    return self.distance_fn(away_blade, home_chest).pow(2) - self.distance_fn(home_chest, away_blade).pow(2)

  def forward(self, x_home, x_away):
    h_home = self.feature_bn(x_home)
    h_away = self.feature_bn(x_away)

    home_blade = self.blade_transfrom(h_home)
    home_blade = self.hidden_bn(home_blade)

    home_chest = self.chest_transform(h_home)
    home_chest = self.hidden_bn(home_chest)

    away_blade = self.blade_transfrom(h_away)
    away_blade = self.hidden_bn(away_blade)

    away_chest = self.chest_transform(h_away)
    away_chest = self.hidden_bn(away_chest)

    matchup = self._matchup_score(away_blade, home_chest, home_blade, away_chest).view(-1, 1)
    result_h = self.result_transform(matchup)

    return self.classifier(result_h)

In [161]:
train_size = 3200
validation_size = 400
batch_size = 32
rand_indcs = torch.randperm(results.shape[0])
train_indcs = rand_indcs[:train_size]
val_indcs = rand_indcs[train_size: train_size + validation_size]
test_indcs = rand_indcs[train_size + validation_size:]
train_home_features = home_features[train_indcs, :].reshape(batch_size, -1, away_features.shape[-1])
train_away_features = away_features[train_indcs, :].reshape(batch_size, -1, away_features.shape[-1])
train_results = results[train_indcs].reshape(batch_size, -1)

val_home_features = home_features[val_indcs, :]
val_away_features = away_features[val_indcs, :]
val_results = results[val_indcs]
test_home_features = home_features[test_indcs, :]
test_away_features = away_features[test_indcs, :]
test_results = results[test_indcs]

In [142]:
def train(model, home, away, results, optimizer, criterion):
  model.train()
  optimizer.zero_grad()
  output = model(home, away)
  loss = criterion(output, results)
  loss.backward()
  optimizer.step() 
  return loss.item()


@torch.no_grad()
def eval(model, home, away, results):
  model.eval()
  output = model(home, away).argmax(-1)
  correct = (output == results).sum().item()
  return (correct / results.shape[0])

In [175]:
bc_model = BladeChest(home_features.shape[-1], 32).double()
criterion = NLLLoss()
print(f'Initial Random State Model: ')
print(f'Validation Acc: {eval(bc_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
print(f'Test Acc: {eval(bc_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Initial Random State Model: 
Validation Acc:  33.0000%
Test Acc:  27.5362%


In [181]:
optimizer = Adam(bc_model.parameters(), lr=1e-3, weight_decay=0)
try:
  for epoch in range(1500000):
    b_loss = 0
    for home, away, result in zip(train_home_features, train_away_features, train_results):
      b_loss += train(bc_model, home, away, result, optimizer, criterion)
    print(f'Epoch: {epoch + 1}')
    print(f'Batch Avg. Loss: {b_loss / batch_size: .4f}')
    print(f'Validation Acc: {eval(bc_model, val_home_features, val_away_features, val_results) * 100: .4f}%')
    print('=' * 64)
except KeyboardInterrupt:
  print(f'Test Acc: {eval(bc_model, test_home_features, test_away_features, test_results) * 100: .4f}%')

Epoch: 1
Batch Avg. Loss:  0.9941
Validation Acc:  51.0000%
Epoch: 2
Batch Avg. Loss:  0.9941
Validation Acc:  50.7500%
Epoch: 3
Batch Avg. Loss:  0.9941
Validation Acc:  50.5000%
Epoch: 4
Batch Avg. Loss:  0.9941
Validation Acc:  50.5000%
Epoch: 5
Batch Avg. Loss:  0.9941
Validation Acc:  50.5000%
Epoch: 6
Batch Avg. Loss:  0.9941
Validation Acc:  51.0000%
Epoch: 7
Batch Avg. Loss:  0.9941
Validation Acc:  51.0000%
Epoch: 8
Batch Avg. Loss:  0.9941
Validation Acc:  51.0000%
Epoch: 9
Batch Avg. Loss:  0.9941
Validation Acc:  51.0000%
Epoch: 10
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 11
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 12
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 13
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 14
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 15
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 16
Batch Avg. Loss:  0.9941
Validation Acc:  51.2500%
Epoch: 17
Batch Avg. Loss:  0.994